<a href="https://colab.research.google.com/github/InbumS/DreamscomeTrue/blob/main/pyspark_pratice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 47.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=fb0ade44bc7da899c8c94f5f376bb759a9c0309f6d76d4a4db3d89065b33f9ce
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf


-------------------------------------------------------------------------------


In [ ]:
spark=SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
from operator import add
lines=spark.read.text('word.txt').rdd.map(lambda r:r[0])
counts=lines.flatMap(lambda x:x.split(' ')).map(lambda x: (x,1)).reduceByKey(add)
output=counts.collect()
for(word,count)in output:
  print("%s: %i"%(word,count))




spark.stop()

## 2022-09-19

* 분산 가능한 데이터 셋(RDD) 
  
* 여러 분산 노드에 나누어 저장

* 변경이 불가 --> 즉 다시 재 생성하는 방법으로 대처 

* RDD에는 크게 두가지 연산(Transformation, Action)

* RDD는 스키마리스 구조 -> 거의 모든 데이터 구조를 섞을 수 있음

로컬 모드 VS 클러스터 모드

여러개의 worker노드 중 하나 실행 -> (클러스터)




In [ ]:
sc = pyspark.SparkContext()
spark=SparkSession.builder.getOrCreate()

In [ ]:
data=sc.parallelize([('Amber',22),('Alfred',23),('Skye',4),('Albert',12),('Amber',20)])

data.collect()

In [ ]:
data_heterogenous=sc.parallelize([
    ('Ferrari','fast'),{'Porshe':100000},['Spain','visited',4504]]) .collect()


In [ ]:
data_heterogenous[2]

In [ ]:
# iris데이터를 webget명령어로 다운받는 경우
!wget https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv

In [ ]:
# 외부 데이터를 읽으려면 Sparkcontext함수의 textFile문법으로 읽는다
data_from_file=sc.textFile('iris.csv')

In [ ]:
data_from_file.take(5)

In [ ]:
iris_header=data_from_file.first()

# RDD화를 꼭 시켜줘야된다.
iris_header=sc.parallelize([iris_header])

iris=data_from_file.subtract(iris_header)
iris.collect()[:10] # Index processing

In [ ]:
iris_parsed_1=iris.map(lambda row: row.split(","))
iris_parsed_1.take(10)

In [ ]:
iris_parsed_2=iris_parsed_1.map(lambda x: (str(x[-1]),float(x[0])))
iris_parsed_2.take(10)

In [ ]:
print(iris_parsed_1.count())

iris_filterd=iris_parsed_1.filter(lambda x:float(x[0])>=5.0)

iris_filterd.count()

In [ ]:
iris_flat=iris_parsed_1.flatMap(lambda x: (str(x[-1]),float(x[0])))
iris_flat.take(5)

In [ ]:
distinct_iris=iris_parsed_1.map(lambda x:x[-1]).distinct().collect()
distinct_iris

## 2022-09-21

* join연산에서는 join attribute가 필요하다

* partition 수가 달라지면 reduce 연산 값이 달라진다

* saveAsTextFile는 같은 디렉토리에 파티션 단위로 파일을 출력

In [ ]:
fraction=0.5
iris_sample1=iris_parsed_1.sample(False,fraction,123)
iris_sample2=iris_parsed_1.sample(False,fraction,123)
#3번째 인자 seed 값을 다르게 쓰면 다르게 sampling 된다.
iris_sample3=iris_parsed_1.sample(False,fraction,456)

iris_sample1.take(5)

In [ ]:
print(iris_sample2.take(5))
print(iris_sample3.take(5))

print("original dataset: {0},sample{1}".format(iris_parsed_1.count(),iris_sample1.count()))

In [ ]:
rdd1=sc.parallelize([('a',1),('b',4),('c',10)])
rdd2=sc.parallelize([('a',4),('a',1),('b','6'),('d',15)])


rdd3=rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

In [ ]:
rdd4=rdd1.join(rdd2)
rdd4.collect()

In [ ]:
len(rdd1.glom().collect())

rdd1=rdd1.repartition(1)
len(rdd1.glom().collect())

In [ ]:
data_reduce=sc.parallelize([1,2,.5,.1,5,.2],1)
data_reduce.reduce(lambda x,y: x/y)

In [ ]:
data_reduce=sc.parallelize([1,2,.5,.1,5,.2],8)
data_reduce.reduce(lambda x,y:x / y)

mapValues()=>
key처리가 필수적이으나 key는 그냥 두고 value만 처리

In [ ]:
iris_parsed_2=iris_parsed_1.map(lambda x:(str(x[-1]),float(x[0])))
iris_parsed_2.take(10)

In [ ]:
iris_totals=iris_parsed_2.mapValues(lambda x:(x,1))
iris_totals.take(10)


In [ ]:
iris_totals=iris_totals.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))
iris_totals.collect()

In [ ]:
iris_means=iris_totals.mapValues(lambda x: (x[0]/x[1]))
result=iris_means.collect()
result

In [ ]:
# f formating 방법
for val in result:
  print(f"iris species {val[0]} has an average sepal length of {val[1]}")